In [15]:
import folium
import branca.colormap as cm
import numpy as np
import pandas as pd

def generate_aadt_map_point(csv_file):
    """
    Generate a scatter map of AADT (Annual Average Daily Traffic) with colors varying by AADT value based on a CSV file.
    
    Parameters:
    - csv_file (str): Path to the CSV file, which must contain the columns "Latitude", "Longitude", and "VEHICLE_AADT_TOTAL".
    
    Returns:
    - folium.Map object
    """
    # 1. Read the CSV file
    df = pd.read_csv(csv_file)
    filtered_df = df[df["CNTY"].isin(["SD", "LA", "ORA"])].copy()

    # Calculate the difference in POSTMILE values
    filtered_df["POSTMILE_DIFF"] = filtered_df["POSTMILE"].diff()

    # Keep only rows where the difference is >= 2
    filtered_df = filtered_df[filtered_df["POSTMILE_DIFF"] >= 2]
    
    # 2. Create a Folium map (set the center to the mean of coordinates)
    m = folium.Map(
        location=[filtered_df["Latitude"].mean(), filtered_df["Longitude"].mean()], 
        zoom_start=7, 
        tiles="OpenStreetMap"
    )

    # 3. Create a color map (for the scatter plot with a gradient of AADT values)
    colormap = cm.LinearColormap(
        colors=["blue", "green", "yellow", "red"],  
        vmin=filtered_df["VEHICLE_AADT_TOTAL"].min(),  
        vmax=filtered_df["VEHICLE_AADT_TOTAL"].max()
    )

    # 4. Loop through the data and plot the scatter points
    for _, row in filtered_df.iterrows():
        lat, lon, aadt = row["Latitude"], row["Longitude"], row["VEHICLE_AADT_TOTAL"]
        color = colormap(aadt) # Calculate color based on AADT value

        folium.CircleMarker(
            location=(lat, lon),
            radius=4,  # Marker size
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.8,
            popup=folium.Popup(f"AADT: {aadt}", parse_html=True)  # Display AADT value on hover
        ).add_to(m)

    # 5. add legend to the map
    colormap.caption = "AADT traffic data"
    colormap.add_to(m)

    # 6. Return the map object
    return m


In [16]:
aadt_map = generate_aadt_map_point("D:/143/project/coordinates/i5_2022_coordinates.csv")  

display(aadt_map)